## Differential_SNP_Determination

##### Marc Bañuls Tornero
---

In [1]:
# Modules needed for the analysis
import numpy as np
import pandas as pd
import io
import os

### Introduction
The initial files are 26 Variant Call Format files. The expected output is a table containing in one column all the positions found in the vcf files provided, and one column for each isolate indicating presence/absence of SNPs at each position.

### Methodology

To do this, the main tools used will be Python code and the two most used data science libraries in Python: Numpy and Pandas.

The procedure designed to obtain the expected output is:
- Design a function to open vcf files and read them without its metadata and as Pandas DataFrames
- Create a Pandas DataFrame with the unique SNPs positions of all vcf files, including the information of its Position (POS), Reference base (REF) and Alternative base (ALT).
- Compare the unique positions, Ref and Alt bases of each vcf file with the respective ones in the DataFrame with the unique SNPs. Then add to the DataFrame a column for each vcf file with the results. If the unique position is found in the isolate, it will be marked as a "1". Otherwise it will be a "0".
- Determine differential SNPs of each strain

#### - Design a function to open vcf files
The vcf files have been uploaded to the folder 'vcf_files'. After that, defining a function will be helpful to open the vcf files and read them as Pandas DataFrames.

In [2]:
def read_vcf(path):
    # Open the vcf file and read only the lines that
    # do not contain metadata (marked with two hash marks "##")
    lines = []
    with open(path, "r") as f:
        for line in f:
            if not line.startswith("##"):
                lines.append(line)
    
    # The output desired is a Pandas DataFrame:
    return pd.read_csv(
        # io.StringIO is used to read the data in lines
        io.StringIO(''.join(lines)),
        sep='\t'
    # Hash removal in header
    ).rename(columns={'#CHROM': 'CHROM'})

To be able to use the **read_vcf** function it's needed the path of the vcf files and their names easily callable. To do this, a variable with the full path and a list with all the names of the vcf files are created. Also, it's advisable to sort the filenames list for better readability and organization.

In [3]:
path = "/home/vcf_files/"
filenames = sorted(os.listdir(path))

Now with the path, names and function, the files are easily callable and readable. 

#### - Create a DataFrame with unique positions, Ref and Alt bases

To create a Pandas DataFrame with all the possible SNPs (including its position, Ref and Alt bases), we create three lists. In **pos_list** there will be all the positions of the SNPS, in **ref_list** all the Reference bases for each SNPs, and in **alt_list** all its respective alternative bases. Then we iterate through all the vcf files and append to each list its respective data (positions, ref and alt bases).

In [4]:
pos_list = []
ref_list = []
alt_list = []
for name in filenames:
    fullpath = path + name
    sample = read_vcf(fullpath)
    # The SNP positions are in the column "POS" of the vcf file 
    for i in sample.POS:
        pos_list.append(i)
    
    # In the column "REF" are the Reference bases
    for i in sample.REF:
        ref_list.append(i)
    
    # In the column "ALT" are the Alternative bases
    for i in sample.ALT:
        alt_list.append(i)

Now with all the lists we can create a Pandas DataFrame with all the unique SNPs, using the method *drop_duplicates()* from the Pandas library. In this DataFrame we have in the first column the Position, in the second column the Reference base and in the third column the Alternative base of each SNP. This DataFrame will be called **df_snps**.

In [5]:
df_snps = pd.DataFrame(
{"POS": pos_list,
"REF": ref_list,
 "ALT": alt_list
}).drop_duplicates()

#### - Compare the unique positions of the DataFrame with the positions in each vcf file

Now we have to add a column representing each isolate (a total of 26 columns) that informs if the specific isolate has a SNP or not in each position using the "POS", "REF" and "ALT" columns of **df_snps** as reference. To do so, we iterate through the 26 vcf files and read them as Pandas DataFrames (with the function **read_vcf** previously defined). After that, we create an empty list **list_positions** that will be used to store the results checked for each file separately.

For each file we iterate through each row of the **df_snps** DataFrame and we check if the unique SNP exists in the in the vcf file. To do so we check if the Position, Reference and Alternative base are the same in the position of the vcf file checked. If the unique position exist in the file, a "1" is appended to **list_positions**. Otherwise, a 0 is appended to the list. As we are doing this using iterations, the created list is correctly ordered with the unique positions that are in the Pandas DataFrame.

When all the unique positions are checked, the list is added as a column to the **df_snps** DataFrame, with the name of the file as a header (to improve readability of the names we cut the part of the name that is the same in all files). Using an iteration is possible to automatically create the named columns for each isolate checked succesfully.

In [6]:
# To iterate through all vcf files (strains):
for name in filenames:
    fullpath = path + name
    isolate = read_vcf(fullpath)
    list_positions = []
    
    # To iterate through all the rows (SNPs):
    for i in range(len(df_snps)):
        # Check if the row with the exact POS, REF and ALT
        # exist in the vcf file of the strain
        if df_snps.iloc[i,0] in isolate.POS.to_numpy() and \
        df_snps.iloc[i,1] in isolate[isolate.POS == df_snps.iloc[i,0]].REF.to_numpy() and \
        df_snps.iloc[i,2] in isolate[isolate.POS == df_snps.iloc[i,0]].ALT.to_numpy():
            list_positions.append(1)
        else:
            list_positions.append(0)
    
    df_snps[name[:-26]] = list_positions

#### Determine differential SNPs

To get the SNPs that only appear in one strain of all the vcf files (differential SNPs) we can create a separate Pandas DataFrame with them. To fill the DataFrame we will create the lists **pos_unique**, **ref_unique**, **alt_unique** and **name_unique** to store the information.

To check if a SNP from **df_snps** appear only in one vcf file (one strain) we can sum all the strain columns iterating for each row (that contains a unique SNP). If the sum gives the value of "1" it means that only one strain has a 1 and all the others have a 0 in that SNP, so the SNP only appears in one strain (differential).

Now to find which strain has the specific SNP we iterate through all the columns of its row until a "1" is found. When the strain is found, we can append the information of the SNP and the name of the column to its respective created lists.

In [7]:
# Empty list creation:
pos_unique = []
ref_unique = []
alt_unique = []
name_unique = []

# Iterate through df_snps rows:
for i in range(len(df_snps)):
    
    # Do the sum from the 4rd column onwards (where 
    # the strain columns start) 
    if df_snps.iloc[i,3:].sum() == 1:
        
        # Iterate through the columns:
        for j in range(len(df_snps.columns)):
            
            # To find the differential column:
            if df_snps.iloc[i,j] == 1:
                
                # Append the row values of the SNP and
                # the column name:
                pos_unique.append(df_snps.iloc[i,0])
                ref_unique.append(df_snps.iloc[i,1])
                alt_unique.append(df_snps.iloc[i,2])
                name_unique.append(df_snps.columns[j])
            

Lastly, with all the information stored in the lists, we can create a Pandas DataFrame with all the differential SNPs.

In [8]:
df_unique_snps = pd.DataFrame(
{"POS": pos_unique,
"REF": ref_unique,
 "ALT": alt_unique,
 "NAME": name_unique
})

### Results

At this point we have a Pandas DataFrame named as **df_snps** that contains information of all the unique SNPs with the strains as columns to see the presence/absence of the specific SNP in each strain, and a Pandas DataFrame named as **df_unique_snps** that contains information only of differential SNPs and in which strains appear. Lastly we can save these Dataframe obtained as a tsv file.

In [9]:
# Index set to False to improve readability in file
df_snps.to_csv('df_snps.tsv', sep = '\t', index=False)
df_unique_snps.to_csv('df_unique_snps.tsv', sep = '\t', index=False)


---

Thank you for checking this test. If it was interesting/helpful or you have ideas to improve the code do not hesitate in contacting me.